# Running LLaMa from a notebook.

To get started, start up your Ray cluster if it has not already been created:
```
ray up -y cluster/dev.yaml
```

You can monitor the status of the Ray cluster as follows:
```
ray monitor cluster/dev.yaml
```

Once the cluster is up, you'll need to connect ot the Ray cluster. For simplicity, you can run:
```
./scripts/get_cluster_ip.sh
```

which will print something like this:
```
$ ./get_cluster_ip.sh 
External IP address of the head node: 35.186.77.141
Internal IP address of the head node: 10.130.0.107


If developing from a notebook, connect to the Ray cluster as follows:

import ray

ray.init("ray://10.130.0.107:10001")
```


In [ ]:
import ray

runtime_env = {
  "working_dir": "../train",
}

IP = "" # Fill this out
ray.init(f"ray://{IP}:10001", runtime_env=runtime_env)

The following code runs a simple LLaMa 2B training model on a single host TPU. 

In [ ]:
@ray.remote(resources={"TPU": 4})
def init_and_run_hf():
  from llama_hf import init_hf, train
  # hf = Huggingface initialized objects, e.g.
  # datasets, tokenizer, trainer, etc.
  hf = init_hf(model_name="llama2/2B")

  return train(
      training_args=hf["training_args"],
      data_args=hf["data_args"],
      trainer=hf["trainer"],
      train_dataset=hf["train_dataset"])


In [ ]:
try:
  ray.get(init_and_run_hf.remote())
except Exception as e:
  print(f"Caught failure {e}. Shutting down")
  ray.shutdown()